# Agrovita Report


In [ ]:
# Este código está diseñado para ejecutarse en un notebook donde ya tienes el DataFrame procesado (por ejemplo, df o df_season).
# Solo debes copiar y pegar este bloque en una celda del notebook, ajustando el nombre del DataFrame si es necesario.

import pandas as pd
import os

# Si tu DataFrame procesado se llama distinto, reemplaza 'df' por el nombre correcto (por ejemplo, df_season)
df_agrovita = df[df['Exporter Clean'] == 'Agrovita'].copy()
agrovita_lotids = df_agrovita['Lotid'].unique().tolist()

# 1. Initial Stock
initial_stock = analyze_initial_stock_by_lotid(df)
initial_stock = initial_stock[initial_stock['Lotid'].isin(agrovita_lotids)]

# 2. Sales Detail
sales_detail = analyze_sales_detail_by_lotid_and_exporter(df)
sales_detail = sales_detail[sales_detail['Lotid'].isin(agrovita_lotids)]

# 3. Inventory Movements
inventory_movements = analyze_inventory_by_exporter_fifo(df)
inventory_movements = inventory_movements[inventory_movements['Lotid'].isin(agrovita_lotids)]

# 4. Inventory Summary by Lotid
inventory_summary = inventory_movements.groupby('Lotid').agg(
    Initial_Stock=('Initial Stock', lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce').sum()),
    Sales_Quantity=('Sale Quantity', lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce').sum()),
    Current_Inventory=('Current Inventory', lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce').max())
).reset_index()

# 5. Deductions vs Initial Stock
deductions_initial_stock = analyze_deductions_initial_stock(df)
deductions_initial_stock = deductions_initial_stock[deductions_initial_stock['Lotid'].isin(agrovita_lotids)]

# 6. Deductions vs Sales
deductions_sales = analyze_deductions_exporter_view(df, sales_detail)
deductions_sales = deductions_sales[deductions_sales['Lotid'].isin(agrovita_lotids)]

# 7. Export all to Excel in the reports folder
reports_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'reports'))
os.makedirs(reports_dir, exist_ok=True)
excel_path = os.path.join(reports_dir, 'Agrovita Report.xlsx')

with pd.ExcelWriter(excel_path) as writer:
    initial_stock.to_excel(writer, sheet_name='Initial Stock', index=False)
    sales_detail.to_excel(writer, sheet_name='Sales Detail', index=False)
    inventory_movements.to_excel(writer, sheet_name='Inventory Movements', index=False)
    inventory_summary.to_excel(writer, sheet_name='Inventory Summary', index=False)
    deductions_initial_stock.to_excel(writer, sheet_name='Deductions Initial Stock', index=False)
    deductions_sales.to_excel(writer, sheet_name='Deductions Sales', index=False)

print(f"Report generated: {excel_path}")